In [1]:
import cv2
import mediapipe as mp
import numpy as np
import networkx as nx
import plotly.graph_objects as go

## Setup MediaPipe


In [2]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

def create_3d_graph():
    G = nx.random_geometric_graph(10, 0.5, dim=3)
    pos = nx.get_node_attributes(G, 'pos')
    for k, v in pos.items():
        G.nodes[k]['pos'] = np.array(v)
    return G


## Inicia câmera e manipulação

In [8]:
cap = cv2.VideoCapture(0)
prev_x, prev_y = None, None
camera_eye = dict(x=1.25, y=1.25, z=1.25)
G = create_3d_graph()


while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        index_finger = hand_landmarks.landmark[8]  # dedo indicador

        h, w, _ = frame.shape
        x = int(index_finger.x * w)
        y = int(index_finger.y * h)

        if prev_x is not None and prev_y is not None:
            dx = (x - prev_x) / 100.0
            dy = (y - prev_y) / 100.0
            camera_eye['x'] += dx
            camera_eye['y'] -= dy

        prev_x, prev_y = x, y

        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cv2.imshow("Camera - Controle do grafo - Pressione 'q' para sair.", frame)
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
